In [3]:
conda install python = 3.5.0


CondaValueError: invalid package specification: =


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install --user --upgrade "git+https://github.com/IBMStreams/streamsx.health.git#egg=healthdemo&subdirectory=samples/HealthcareJupyterDemo/package"


  Cloning https://github.com/IBMStreams/streamsx.health.git to /private/var/folders/x3/v1b923b96hng5mn1ssc4k27r0000gn/T/pip-install-l4x0r7rb/healthdemo
  Stored in directory: /private/var/folders/x3/v1b923b96hng5mn1ssc4k27r0000gn/T/pip-ephem-wheel-cache-rwuqiziw/wheels/8c/f4/21/deca69ad13332ada14e4e5c4e4f41bc377e7d1e63b58c103ec
Successfully built healthdemo
  Found existing installation: healthdemo 1.0
    Uninstalling healthdemo-1.0:
      Successfully uninstalled healthdemo-1.0


In [5]:
pip install streamsx

Note: you may need to restart the kernel to use updated packages.


In [6]:
import json
import getpass
from streamsx.topology.context import ConfigParams
credentials=getpass.getpass('Streaming Analytics credentials:')

vs={'streaming-analytics': [{'name': 'Streaming Analytics', 'credentials': json.loads(credentials)}]}
config={ConfigParams.VCAP_SERVICES: vs, ConfigParams.SERVICE_NAME: 'Streaming Analytics'}

Streaming Analytics credentials:········


In [8]:
from streamsx.topology import schema
from streamsx.topology.topology import Topology
from streamsx.topology.context import submit

## The healthdemo package provides tools to analyze patient data
## See https://github.com/IBMStreams/streamsx.health/tree/develop/samples/HealthcareJupyterDemo/package
from healthdemo.patientmonitoring_functions import streaming_rpeak
from healthdemo.healthcare_functions import GenTimestamp, aggregate
from healthdemo.windows import SlidingWindow

topo = Topology('HealthcareDemo')

## The ingest-physionet provides data at a rate of 125 tuples/sec
sample_rate = 125

## Subscribe to the topic
patients_data = topo.subscribe('ingest-physionet', schema.CommonSchema.Json)
            
## Add timestamp to the data, so you can perform windowing
patients_data = patients_data.transform(GenTimestamp(sample_rate))

## Generate a window based on the provided sample_rate
patients_data = patients_data.transform(SlidingWindow(length=sample_rate, trigger=sample_rate-1))

## Aggregate the data within the window and create a tuple
patients_data = patients_data.transform(aggregate)

## Process data from 'ECG Lead II' and calculate RPeak and RR delta
patients_data = streaming_rpeak(patients_data, sample_rate, data_label='ECG Lead II')

## Create view for viewing patient vital data
patients_vital = patients_data.view(name='patients_vital')

rc = submit('STREAMING_ANALYTICS_SERVICE', topo, config)
print ("DONE")

RuntimeError: The ANALYTICS_SERVICE context only supports Python version 3.5